<a href="https://colab.research.google.com/github/anjogu/Data_science/blob/building_pipelines_with_python/%5BSolution_Notebook%5D_AfterWork_Introduction_to_Data_Engineering_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Solution Notebook] AfterWork: Introduction to Data Engineering with Python

# Pre-requisites

In [ ]:
# Import pandas as pd
import pandas as pd

# 1. Simple Data Pipeline

In [ ]:
# Define the extract function
def extract():
    name = "Jane"
    return name

# Define the transform function
def transform(data):
    return data

# Define the load function
def load(data):
    print('First name:', data)

# Create the data pipeline
def data_pipeline():
    data = extract()
    transformed_data = transform(data)
    load(transformed_data)

# Execute the data pipeline
data_pipeline()

### <font color="green">Challenge</font>

Imagine you are tasked with creating a basic data pipeline to process and display information about a person's email address.

* Extract Function: Implement extract() to return the email address "jane.doe@example.com".
* Transform Function: Implement transform(data) to convert the email address data to lowercase.
* Load Function: Implement load(data) to print "Person's email address: " followed by data.

* Data Pipeline: Implement data_pipeline() to:
  * Call extract() to obtain the person's email address.
  * Call transform() with the extracted email address.
  * Call load() to display the transformed email address.

* Execute the Data Pipeline: Call data_pipeline() to execute the data pipeline.

In [ ]:
# Define the extract function
def extract():
    return "jane.doe@example.com"

# Define the transform function
def transform(data):
    return data.lower()

# Define the load function
def load(data):
    print('Person\'s email address:', data)

# Create the data pipeline
def data_pipeline():
    data = extract()
    transformed_data = transform(data)
    load(transformed_data)

# Execute the data pipeline
data_pipeline()

# 2. CSV Data Pipeline

In [ ]:
# Define the extract function
def extract():
    employees = pd.read_csv('employees.csv')
    departments = pd.read_csv('departments.csv')
    return employees, departments

# Define the transform function
def transform(employees, departments):
    merged_data = pd.merge(employees, departments, on='department_id')
    return merged_data

# Define the load function
def load(data):
    data.to_csv('merged_data.csv', index=False)

# Create the data pipeline
def data_pipeline():
    employees, departments = extract()
    merged_data = transform(employees, departments)
    load(merged_data)

# Execute the data pipeline
data_pipeline()

### <font color="green">Challenge</font>

Carry out ETL on the [movie_ratings.csv](https://bit.ly/3yNFJYx) dataset provided. Some actions you could carry out in the transform stage include:
- Calculating the average rating for each movie.
- Calculating the number of ratings per user.
- Grouping movies into rating categories (e.g., high-rated, low-rated).

In [ ]:
# Extract function
def extract():
    patients = pd.read_csv('https://bit.ly/3yNFJYx')
    return patients

# Transform function
def transform(patients):

    # Remove missing values and duplicates
    patients.dropna(inplace=True)
    patients.drop_duplicates(inplace=True)

    # Encode the Gender column
    patients['Gender'] = patients['Gender'].map({'Male': 0, 'Female': 1, 'Other': 2})

    # Convert Date of Birth, Admission Date, and Discharge Date to datetime. Also calculate age and create
    patients['Date of Birth'] = pd.to_datetime(patients['Date of Birth'])
    patients['Admission Date'] = pd.to_datetime(patients['Admission Date'])
    patients['Discharge Date'] = pd.to_datetime(patients['Discharge Date'])
    patients['Age'] = (patients['Admission Date'] - patients['Date of Birth']).dt.days // 365

    # Calculate the length of hospital stay
    patients['Length of Stay'] = (patients['Discharge Date'] - patients['Admission Date']).dt.days

    # Extract Blood Pressure and Pulse from Vital Signs
    patients[['Blood Pressure', 'Pulse']] = patients['Vital Signs'].str.extract(r'Blood Pressure: (\d+/\d+), Pulse: (\d+)')
    patients.drop('Vital Signs', axis=1, inplace=True)

    # Aggregate data monthly and yearly
    patients['Year'] = patients['Admission Date'].dt.year
    patients['Month'] = patients['Admission Date'].dt.month

    # Grouping
    monthly_aggregation = patients.groupby(['Year', 'Month']).agg({
        'Patient ID': 'count',
        'Age': 'mean',
        'Length of Stay': 'mean'
    })
    yearly_aggregation = patients.groupby(['Year']).agg({
        'Patient ID': 'count',
        'Age': 'mean',
        'Length of Stay': 'mean'
    })

    return patients, monthly_aggregation, yearly_aggregation

# Load function
def load(patients, monthly_aggregation, yearly_aggregation):
    patients.to_csv('transformed_patient_health_records.csv', index=False)
    monthly_aggregation.to_csv('monthly_aggregation.csv')
    yearly_aggregation.to_csv('yearly_aggregation.csv')

# Data pipeline function
def data_pipeline():
    patients = extract()
    patients, monthly_aggregation, yearly_aggregation = transform(patients)
    load(patients, monthly_aggregation, yearly_aggregation)

# Execute the data pipeline
data_pipeline()

# 3. JSON Data Pipeline

In [ ]:
# Extract function (Employees: https://bit.ly/4bSEXrV, Departments: https://bit.ly/3yQaKLu)
def extract():
    employees = pd.read_json('https://bit.ly/4bSEXrV', orient='records')
    departments = pd.read_json('https://bit.ly/3yQaKLu', orient='records')
    return employees, departments

# Transform function
def transform(employees, departments):
    # Merge employees and departments on 'department_id' and 'id'
    merged_data = pd.merge(employees, departments, left_on='department_id', right_on='id', suffixes=('_emp', '_dept'))
    transformed_data = merged_data[['id_emp', 'name_emp', 'age', 'gender', 'department_id', 'name_dept', 'location']]
    transformed_data.columns = ['id', 'name', 'age', 'gender', 'department_id', 'department_name', 'department_location']
    return transformed_data

# Load function
def load(data):
    data.to_csv('transformed_json_data.csv', index=False)

# Create the data pipeline
def data_pipeline():
    employees, departments = extract()
    transformed_data = transform(employees, departments)
    load(transformed_data)

# Execute the data pipeline
data_pipeline()

### <font color="green">Challenge</font>

Create a data pipeline for merging order and product data, calculating derived values, and saving the transformed data into a CSV file.

In [ ]:
# Extract function: (Orders: https://bit.ly/4bKdvwo, Products: https://bit.ly/45eJKkN)
def extract():
    orders = pd.read_json('https://bit.ly/4bKdvwo', orient='records')
    products = pd.read_json('https://bit.ly/45eJKkN', orient='records')
    return orders, products

# Transform function
def transform(orders, products):
    merged_data = pd.merge(orders, products, on='product_id', how='left')
    merged_data['total_price'] = merged_data['quantity'] * merged_data['price']
    transformed_data = merged_data[['order_id', 'product_id', 'name', 'quantity', 'price', 'total_price']]
    transformed_data.columns = ['Order ID', 'Product ID', 'Product Name', 'Quantity', 'Price', 'Total Price']
    return transformed_data

# Load function
def load(data):
    data.to_csv('order_product_details.csv', index=False)

# Create the data pipeline
def data_pipeline():
    orders, products = extract()
    transformed_data = transform(orders, products)
    load(transformed_data)

# Execute the data pipeline
data_pipeline()

# 4. Data Pipeline from Scraped data

In [ ]:
# Importing the required libraries
import requests
from bs4 import BeautifulSoup

In [ ]:
# Extract function
def extract():
    # Send a GET request to the URL and parse the HTML
    response = requests.get("http://books.toscrape.com/")
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all book containers
    books = []
    book_containers = soup.find_all('article', class_='product_pod')

    # Extract title, price_color, and star-rating from each book container
    for container in book_containers:
        title = container.find('h3').a['title']
        price = container.find('p', class_='price_color').text.strip()
        rating = container.find('p', class_='star-rating')['class'][1]
        books.append([title, price, rating])

    return books

# Transform function
def transform(data):
    df = pd.DataFrame(data, columns=['Title', 'Price', 'Rating'])
    df['Price'] = df['Price'].apply(lambda x: float(x.strip('£')))
    df['Rating'] = pd.to_numeric(df['Rating'])
    return df

# Load function
def load(data):
    data.to_csv('books_data.csv', index=False)

# Create the data pipeline
def data_pipeline():
    book_data = extract()
    df = transform(book_data)
    load(df)

# Execute the data pipeline
data_pipeline()

### Challenge

Create a data pipeline that extracts quotes from the [Quotes to Scrape](https://quotes.toscrape.com) website, transforms them, and saves them to a CSV file, we'll modify the existing pipeline accordingly.

In [ ]:
# Extract function
def extract():
    response = requests.get("http://books.toscrape.com/")
    soup = BeautifulSoup(response.text, 'html.parser')
    quotes = []

    # Find all quote containers
    quote_containers = soup.find_all('div', class_='quote')

    # Extract text, author, and tags from each quote container
    for container in quote_containers:
        text = container.find('span', class_='text').text
        author = container.find('small', class_='author').text
        tags = container.find('div', class_='tags').find_all('a')
        tags = [tag.text for tag in tags]
        quotes.append([text, author, tags])

    return quotes

# Transform function
def transform(data):
    # Convert the list of quotes into a DataFrame
    df = pd.DataFrame(data, columns=['Quote', 'Author', 'Tags'])

    # Join tags into a single string separated by commas
    df['Tags'] = df['Tags'].apply(lambda x: ', '.join(x))

    return df

# Load function
def load(data):
      data.to_csv('quotes_data.csv', index=False)

# Create the data pipeline
def data_pipeline():
    quote_data = extract()
    df = transform(quote_data)
    load(df)

# Execute the data pipeline
data_pipeline()